In [6]:
import ccxt
import pandas as pd
from datetime import datetime
from tqdm import tqdm  # Asegúrate de instalarlo: pip install tqdm

# -----------------------------
# BITGET ENTROPIA
# -----------------------------


INITIAL_CAPITAL = 899  # Capital in USDT
# -----------------------------
# CONNECTION
# -----------------------------
import ccxt

def connect_bitget_futures():
    exchange = ccxt.bitget({
        'apiKey': BITGET_API_KEY,
        'secret': BITGET_API_SECRET,
        'password': BITGET_API_PASS,
        'enableRateLimit': True,
    })
    # Indicar que trabajaremos en futuros USDT
    exchange.options['defaultType'] = 'future'  # 'spot' es default
    exchange.load_markets()
    return exchange


# -----------------------------
# FETCH CLOSED ORDERS CON PROGRESO
# -----------------------------
def get_closed_orders(limit=50):
    exchange = connect_bitget()
    all_orders = []

    symbols = list(exchange.markets.keys())

    # Usamos tqdm para la barra de progreso
    for symbol in tqdm(symbols, desc="Fetching closed orders"):
        try:
            closed_orders = exchange.fetch_closed_orders(symbol, limit=limit)

            for o in closed_orders:
                o["symbol"] = symbol
                all_orders.append(o)

        except Exception as e:
            tqdm.write(f"⚠️ Error with {symbol}: {e}")  # Escribe errores sin romper la barra de progreso

    df = pd.DataFrame(all_orders)
    columns = ["symbol", "id", "type", "side", "price", "amount", "status", "datetime"]
    available_cols = [c for c in columns if c in df.columns]

    return df[available_cols] if not df.empty else df


# -----------------------------
# FILTER ORDERS BY DATE
# -----------------------------
def filter_by_date(df, since_date):
    if df.empty:
        return df

    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    if df['datetime'].dt.tz is not None:
        df['datetime'] = df['datetime'].dt.tz_localize(None)
    
    since_dt = pd.to_datetime(since_date)
    df_filtered = df[df['datetime'] > since_dt]
    
    return df_filtered

# -----------------------------
# CALCULATE PROFITABILITY PER SYMBOL
# -----------------------------
def calculate_profitability(df):
    if df.empty:
        print("⚠️ No orders to calculate profitability.")
        return

    summary = []

    for symbol, group in df.groupby("symbol"):
        buys = group[group["side"] == "buy"]
        sells = group[group["side"] == "sell"]

        total_bought = (buys["price"] * buys["amount"]).sum()
        total_sold   = (sells["price"] * sells["amount"]).sum()

        if total_bought > 0:
            profit_percent = (total_sold - total_bought) / total_bought * 100
            summary.append([symbol, len(group), profit_percent])

    df_summary = pd.DataFrame(summary, columns=["Symbol", "Total Orders", "Profitability (%)"])
    print("\n📊 Profitability by symbol:")
    print(df_summary)

    # -----------------------------
    # Calculate positive profitability %
    # -----------------------------
    total_symbols = len(df_summary)
    positive_symbols = (df_summary["Profitability (%)"] > 0).sum()
    positive_percent = (positive_symbols / total_symbols) * 100 if total_symbols > 0 else 0

    print(f"\n🔹 Total symbols  : {total_symbols}")
    print(f"🔹 Symbols with + : {positive_symbols} ({positive_percent:.2f}%)")

# -----------------------------
# GET TOTAL BALANCE IN USDT
# -----------------------------
def get_total_balance_in_usdt(exchange):
    balance = exchange.fetch_balance()
    total_usdt = 0

    for asset, amount in balance['total'].items():
        if amount == 0:
            continue
        if asset == 'USDT':
            total_usdt += amount
        else:
            symbol = f"{asset}/USDT"
            try:
                ticker = exchange.fetch_ticker(symbol)
                total_usdt += amount * ticker['last']
            except Exception as e:
                print(f"⚠️ Could not convert {asset} to USDT: {e}")
                continue

    return total_usdt

# -----------------------------
# MAIN SCRIPT
# -----------------------------
exchange = connect_bitget()
df_orders = get_closed_orders(limit=50)

# Filter orders by date
since_date = "2025-09-20"  # Change this date as needed
df_orders = filter_by_date(df_orders, since_date)

if df_orders.empty:
    print(f"⚠️ No closed orders found after {since_date}.")
else:
    output_file = "bitget_closed_orders_history.xlsx"
    df_orders.to_excel(output_file, index=False, sheet_name="Closed Orders")
    print(f"📂 Closed orders history saved to '{output_file}'")

    # Calculate profitability per symbol
    calculate_profitability(df_orders)

    # Get final total balance in USDT
    final_balance_usdt = get_total_balance_in_usdt(exchange)
    total_profit_percent = (final_balance_usdt - INITIAL_CAPITAL) / INITIAL_CAPITAL * 100

    print(f"\n💰 Total balance : {final_balance_usdt:.2f} USDT")
    print(f"📈 Total return   : {total_profit_percent:.2f}%")

Fetching closed orders: 100%|██████████| 1453/1453 [06:58<00:00,  3.47it/s]


📂 Closed orders history saved to 'bitget_closed_orders_history.xlsx'

📊 Profitability by symbol:
                   Symbol  Total Orders  Profitability (%)
0    1000000MOG/USDT:USDT             4          -0.034953
1     10000ELON/USDT:USDT             8          -0.275459
2       1000XEC/USDT:USDT            10          -0.082900
3         1INCH/USDT:USDT             2          -0.078616
4    1MBABYDOGE/USDT:USDT             2           0.149577
..                    ...           ...                ...
194         YGG/USDT:USDT             4           0.222528
195     ZEREBRO/USDT:USDT             4           0.024958
196        ZETA/USDT:USDT             2           0.000000
197         ZRC/USDT:USDT             3           0.313435
198         ZRX/USDT:USDT             2          -0.077042

[199 rows x 3 columns]

🔹 Total symbols  : 199
🔹 Symbols with + : 73 (36.68%)
⚠️ Could not convert BAKE to USDT: bitget does not have market symbol BAKE/USDT
⚠️ Could not convert 0X0 to USDT: bi